In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [16]:
url = "http://quotes.toscrape.com"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")


In [17]:
quote = soup.find("div", class_="quote")

quote_text = quote.find("span", class_="text").get_text()
author = quote.find("small", class_="author").get_text()
tags = [tag.get_text() for tag in quote.find_all("a", class_="tag")]

In [18]:
print("Quote:", quote_text)
print("Author:", author)
print("Tags:", tags)

Quote: “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Author: Albert Einstein
Tags: ['change', 'deep-thoughts', 'thinking', 'world']


In [19]:
author_link = quote.find("a")["href"]

author_page = requests.get(url + author_link)
author_soup = BeautifulSoup(author_page.content, "html.parser")


In [20]:
author_born = author_soup.find("span", class_="author-born-date").get_text()
author_born_location = author_soup.find("span", class_="author-born-location").get_text()
description = author_soup.find("div", class_="author-description").get_text(strip=True)

In [21]:
print("Born:", author_born)
print("Born Location:", author_born_location)
print("Description:", description)

Born: March 14, 1879
Born Location: in Ulm, Germany
Description: In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining the photoelectric effect, the basis of electronics, earned him the Nobel Prize in 1921. His first paper on Special Relativity Theory, also published in 1905, changed the world. After the rise of the Nazi party, Einstein made Princeton his permanent home, becoming a U.S. citizen in 1940. Einstein, a pacifist during World War I, stayed a firm proponent of social justice and responsibility. He chaired the Emergency Committee of Atomic Scientists, which organized to alert the public to the dangers of atomic warfare.At a symposium, he advised: "In their struggle for the ethical good, teachers of religion must have the stature to give up the doctrine of a personal God, that is, give up that source of fear and hope which in the past placed such vast power in the hands of priests. In their labor

In [22]:
all_quotes = []
page_1 = "/page/1/"
while page_1:
    response = requests.get(url + page_1)
    soup = BeautifulSoup(response.content, "html.parser")
    quotes = soup.find_all("div", class_="quote")
    for quote in quotes:
        quote_text = quote.find("span", class_="text").get_text()
        author = quote.find("small", class_="author").get_text()
        tags = [tag.get_text() for tag in quote.find_all("a", class_="tag")]
        author_link = quote.find("a")["href"]
        author_page = requests.get(url + author_link)
        author_soup = BeautifulSoup(author_page.content, "html.parser")
        author_born = author_soup.find("span", class_="author-born-date").get_text()
        author_born_location = author_soup.find("span", class_="author-born-location").get_text()
        description = author_soup.find("div", class_="author-description").get_text(strip=True)
        all_quotes.append({"quote": quote_text, "author": author, "tags": ", ".join(tags), "born": author_born, "born_location": author_born_location, "description": description})
    next_page = soup.find("li", class_="next")
    page_1 = next_page.find("a")["href"] if next_page else None



In [23]:
df = pd.DataFrame(all_quotes)
df.to_csv("quotes_scraped.csv", index=False)

In [24]:
df

,quote,author,tags,born,born_location,description
0,“The world as we have created it is a process ...,Albert Einstein,"change, deep-thoughts, thinking, world","March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities, choices","July 31, 1965","in Yate, South Gloucestershire, England, The U...",See also: Robert GalbraithAlthough she writes ...
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational, life, live, miracle, miracles","March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy, books, classic, humor","December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",Jane Austen was an English novelist whose work...
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself, inspirational","June 01, 1926",in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...
...,...,...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,better-life-empathy,"April 28, 1926","in Monroeville, Alabama, The United States","Harper Lee, known as Nelle, was born in the Al..."
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"books, children, difficult, grown-ups, write, ...","November 29, 1918","in New York City, New York, The United States",Madeleine L'Engle was an American writer best ...
97,“Never tell the truth to people who are not wo...,Mark Twain,truth,"November 30, 1835","in Florida, Missouri, The United States","Samuel Langhorne Clemens, better known by his ..."
98,"“A person's a person, no matter how small.”",Dr. Seuss,inspirational,"March 02, 1904","in Springfield, MA, The United States",Theodor Seuss Geisel was born 2 March 1904 in ...
